In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, TimestampType, StringType, DoubleType, LongType
from pyspark.sql.functions import udf, from_json, col, from_unixtime

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'

In [3]:
spark = (
    SparkSession.builder
    .appName('5003-project')
    .master('spark://spark-master:7077')
    .config("spark.files.overwrite", "true")
    .enableHiveSupport()
    .getOrCreate()
)
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/envs/5003-project/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-85dcba24-9cd5-4932-864c-a5e0fd3459cf;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.1-spark3.0-s_2.12!graphframes.jar (1428ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (327ms)
:: resolution report :: resolve 10436ms :: artifacts dl 1793ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	-------------------

In [4]:
spark

In [5]:
from graphframes import *
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [9]:
g.find('(a)-[]->(b)').show()

+----------------+----------------+
|               a|               b|
+----------------+----------------+
|  {d, David, 29}|  {g, Gabby, 60}|
|{c, Charlie, 37}|  {g, Gabby, 60}|
|  {a, Alice, 34}|  {g, Gabby, 60}|
|  {g, Gabby, 60}|  {d, David, 29}|
|  {f, Fanny, 38}|  {d, David, 29}|
| {e, Esther, 32}|  {d, David, 29}|
|{c, Charlie, 37}|  {d, David, 29}|
|  {g, Gabby, 60}|{c, Charlie, 37}|
|  {f, Fanny, 38}|{c, Charlie, 37}|
|    {b, Bob, 36}|{c, Charlie, 37}|
|  {a, Alice, 34}|{c, Charlie, 37}|
|  {f, Fanny, 38}|    {b, Bob, 36}|
|{c, Charlie, 37}|    {b, Bob, 36}|
|  {a, Alice, 34}|    {b, Bob, 36}|
|  {g, Gabby, 60}|  {a, Alice, 34}|
| {e, Esther, 32}|  {a, Alice, 34}|
|  {d, David, 29}|  {a, Alice, 34}|
|{c, Charlie, 37}|  {a, Alice, 34}|
+----------------+----------------+

